In [8]:
import re
import pandas as pd
from datetime import datetime

# f = open('./chats/Dellaware.txt','r',encoding='utf-8')
f = open('./chats/WhatsApp Chat with Squad.exe24.txt','r',encoding='utf-8')
data = f.read()

pattern_24hr = '\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{1,2}\s-\s'
pattern_12hr = '\d{1,2}/\d{1,2}/\d{2,4}, \d{1,2}:\d{1,2}\s[ap][m]\s-\s'
pattern_yr = '\d{1,2}/\d{1,2}/\d{2},'

match_12hr = re.search(pattern_12hr, data)
match_24hr = re.search(pattern_24hr, data)
pattern_year = re.search(pattern_yr,data)

if match_12hr:
    current_time_pattern = '\d{1,2}:\d{1,2}\s[ap][m]'
    current_time_format = '%I:%M %p'
    if pattern_year:
        current_year_pattern = '\d{1,2}/\d{1,2}/\d{2}, '
        pattern = current_year_pattern+current_time_pattern+"\s-\s"
        current_date_format = '%d/%m/%y, '
        date_format = current_date_format+current_time_format
    else:
        year_pattern = '\d{1,2}/\d{1,2}/\d{4}, '
        pattern = year_pattern+current_time_pattern+"\s-\s"
        current_date_format = '%d/%m/%Y, '
        date_format = current_date_format+current_time_format
        
elif match_24hr:
    current_time_pattern = '\d{1,2}:\d{1,2}'
    current_time_format = '%H:%M'
    if pattern_year:
        year_pattern = '\d{1,2}/\d{1,2}/\d{2}, '
        pattern = year_pattern+current_time_pattern+"\s-\s"
        current_date_format = '%d/%m/%y, '
        date_format = current_date_format+current_time_format
    else:
        year_pattern = '\d{1,2}/\d{1,2}/\d{4}, '
        pattern = year_pattern+current_time_pattern+"\s-\s"
        current_date_format = '%d/%m/%Y, '
        date_format = current_date_format+current_time_format
    
date_pattern = year_pattern+current_time_pattern





messages = re.split(pattern,data)[1:]

dates_pre = re.findall(date_pattern, data)

dates = []
for msg in dates_pre:
    date = re.sub(re.escape('\u202f'),' ', msg)
    dates.append(date)
    

Date=[]
for date in dates:
        timestamp = datetime.strptime(date, date_format)
        timestamp_formatted = timestamp.strftime('%d/%m/%Y, %I:%M %p')
        Date.append(timestamp_formatted)
        

df = pd.DataFrame({'user_message': messages, 'message_date': Date})
# convert message_date type

df['message_date'] = pd.to_datetime(df['message_date'], format='%d/%m/%Y, %I:%M %p')


df.rename(columns={'message_date': 'date'}, inplace=True)


users = []
messages = []
for message in df['user_message']:
    entry = re.split('([\w\W]+?):\s', message)
    if entry[1:]:  # user name
        users.append(entry[1])
        messages.append(" ".join(entry[2:]))
    else:
        users.append('group_notification')
        messages.append(entry[0])

df['user'] = users
df['message'] = messages
df.drop(columns=['user_message'], inplace=True)

df['only_date'] = df['date'].dt.date
df['year'] = df['date'].dt.year
df['month_num'] = df['date'].dt.month
df['month'] = df['date'].dt.month_name()
df['day'] = df['date'].dt.day
df['day_name'] = df['date'].dt.day_name()
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute

period = []
for hour in df[['day_name', 'hour']]['hour']:
    if hour == 23:
        period.append(str(hour) + "-" + str('00'))
    elif hour == 0:
        period.append(str('00') + "-" + str(hour + 1))
    else:
        period.append(str(hour) + "-" + str(hour + 1))

df['period'] = period



In [6]:
df

,date,user,message,only_date,year,month_num,month,day,day_name,hour,minute,period
0,2020-10-01 00:58:00,group_notification,Messages and calls are end-to-end encrypted. N...,2020-10-01,2020,10,October,1,Thursday,0,58,00-1
1,2019-05-08 21:54:00,group_notification,"Rohan Sammy created group ""🇮🇳[The Indian Govt....",2019-05-08,2019,5,May,8,Wednesday,21,54,21-22
2,2020-10-01 00:58:00,group_notification,Rohit Sxd added you\n,2020-10-01,2020,10,October,1,Thursday,0,58,00-1
3,2020-10-01 00:59:00,Rohan,bc\n,2020-10-01,2020,10,October,1,Thursday,0,59,00-1
4,2020-10-01 00:59:00,Rohan,🤣\n,2020-10-01,2020,10,October,1,Thursday,0,59,00-1
...,...,...,...,...,...,...,...,...,...,...,...,...
286,2022-04-19 07:58:00,Rohan,<Media omitted>\n,2022-04-19,2022,4,April,19,Tuesday,7,58,7-8
287,2022-04-19 07:59:00,Rohan,@919572060411 @918797902441 @919155936227 @919...,2022-04-19,2022,4,April,19,Tuesday,7,59,7-8
288,2022-04-19 08:16:00,Nemo Dox B.Arch,Haa haa aaenge 😄\n,2022-04-19,2022,4,April,19,Tuesday,8,16,8-9
289,2022-04-19 08:17:00,Rohan,😁DC sahab ni aynge th ksse chlega\n,2022-04-19,2022,4,April,19,Tuesday,8,17,8-9


In [9]:
df['user'].value_counts()

Rohan Deshraj Kujur        3285
Anurag Albert Tigga        2252
Jose Lorenzo Soren         2158
Billincoln Kujur           2007
James Shekhar Lakra         756
Rohan Deshraj Kujur Jio     392
Prince Herenj               268
Milind Richard Horo          75
group_notification           59
Navneet Dungdung             28
Name: user, dtype: int64

In [ ]:
str = "12/12/12, 12:12"
timestamp = datetime.strptime(str,'%d/%m/%y, %H:%M')
print(timestamp)
timestamp_formatted = datetime.strftime(timestamp,'%d/%m/%Y, %I:%M %p')
print(timestamp_formatted)

In [ ]:
a,b,c = "Rohan","is","Good"

In [ ]:
a+b+c